# Newsvendor Models: How Many NFL Replica Jerseys to Order so that Profits Are Maximized?

This notebook shows how to solve the NFL Replica Jerseys / Newsvendor problem in Python from the lectures in MITx's CTL.SC1x: Supply Chain Fundamentals. To take the course, please visit https://www.edx.org/micromasters/mitx-supply-chain-management.

The problem is as follows: In 2002, Reebok had the sole rights to sell NFL football jerseys. Peak sales for the jerseys last about 8 weeks, while the lead time for manufacturing is 12-16 weeks. That means if sales take off in Week 1, it is already too late to order more jerseys. In short, Reebok had to commit to one order in advance, without knowing actual demand and without any ability to course correct after the order was placed.

This is a classic case of the newsvendor problem. Newsvendor models are characterized by probabilistic demand and single period planning horizons. In order to model stochastic demand, we will use SciPy. 

In [1]:
# imports
import numpy as np
from scipy import stats
from scipy.stats import norm

## QUESTION: What Is the Ideal Order Size?

Our variable of interest is Q*, the ideal order size that maximizes expected profits. 

In other words, how many football jerseys should I order, if I know:
* how many jerseys sold historically;
* that demand is stochastic (what demand materialized in the past may not show up again this year); 
* I cannot order extra jerseys if they prove to be popular this year;
* that I still have to pay for every jersey I order, whether they are sold or not.

## Variables

All price units are USD (`$`). History showed demand to be normally distributed with an average of `32,000` jerseys (and a $\sigma$ of `11,000`).

In [2]:
unit_cost = 10.90
unit_price = 24.00
ave_demand = 32000
std_demand = 11000

## How Do We Calculate Profits?

First step is to find the profit. There are only two outcomes: either we overorder the jerseys and demand is less than what we ordered or we don't order enough and there's a shortage. Instead of expressing this in a nice mathematical equation, we will just use a simple function to calculate profits:

In [3]:
def calculate_profit(cost, price, order, demand):
    # outcome 1: didn't order enough
    if order < demand:
        amount = order
    # outcome 2: less demand than expected
    else:
        amount = demand
    return (unit_price * amount) - (unit_cost * order)

The two ways to find ideal order size involve a data table and marginal analysis. I will not replicate here the sprawling Excel data table as a dataframe (although the code below can be used to create it). To warm up, let's start with some of the questions Professor Caplice poses while reviewing the Excel data table.

## Solving Single Period Model: Data Table

### What is my profit if I have a demand of at least 4,000 jerseys and I ordered 25,000 total?

In [4]:
my_profit = calculate_profit(10.90, 24, 25000, 4000)
print("The profit/(loss): {}".format(my_profit)) # 177.000 in Excel (due to rounding)

The profit/(loss): -176500.0


### What is the probability that my demand is going to be 8000 units or less?

In [5]:
# using scipy
my_prob2 = norm.cdf(8000, ave_demand, std_demand)
print("The probability: {}".format(my_prob2)) # 1.5% in Excel

The probability: 0.014561477076192526


In [6]:
# using simulation

# let's take a million samples out of the Normal distribution: samples
samples = np.random.normal(ave_demand, std_demand, size=1000000) # samples is a list of possible demands

# Compute the fraction that are less than or equal to 8000 units: prob
my_prob = np.sum(samples <= 8000)/ len(samples)
print("The probability with simulation: {}".format(my_prob)) # roughly 1.5% again, but bit different

The probability with simulation: 0.014551


## Solving Single Period Model: Marginal Analysis

For marginal analysis, we use two costs: excess cost and shortage costs. They change depending on whether we are calculating costs for the wholesaler, the retailer, or the channel. We will later extend these costs with salvage values and penalties for not meeting demand.

### Marginal analysis for retailer profit without salvage cost or penalty

In [7]:
# shortage cost for now
shortage_cost = unit_price - unit_cost
# excess cost for now
excess_cost = unit_cost
# the critical ratio captures the trade-off
critical_ratio = shortage_cost / (excess_cost + shortage_cost)

In [8]:
# find Q* where the probability of my demand being less or equal to order equals CR
my_q1 = np.ceil(norm.ppf(critical_ratio, ave_demand, std_demand))
print("The ideal order size: {}".format(my_q1)) # should give 33,267

The ideal order size: 33267.0


### Marginal cost for retailer profit with salvage cost

In [9]:
# new variable
salvage_price = 7.00

# shortage cost for now
shortage_cost = unit_price - unit_cost
# excess cost for now
excess_cost_salvage = unit_cost - salvage_price
# the critical ratio captures the trade-off
critical_ratio_salvage = shortage_cost / (excess_cost_salvage + shortage_cost)

In [10]:
# find Q* where the probability of my demand being less or equal to order equals CR
my_q2 = np.ceil(norm.ppf(critical_ratio_salvage, ave_demand, std_demand))
print("The ideal order size with salvage value: {}".format(my_q2)) # should give 40,149

The ideal order size with salvage value: 40149.0


## Single Period Inventory Models: Calculating Expected Profitability

I will not do the calculations for the discrete case ("freshly baked widgets") -- they can be found in the widgets.py document in the repo.

## Expected Units Short

In [11]:
def calculate_expected_units(order_size, mu, sigma):
    k = (order_size - mu)/sigma
    gk = norm.pdf(k, 0, 1) - (k * norm.sf(k))
    exp_us = gk * sigma # this np.ceil needs to be fixed !!!!
    return (exp_us)

To calculate expected units short for normally distributed widgets `~N(160, 45)` and a Q of `190`:

In [12]:
example = calculate_expected_units(190, 160, 45)
print("The expected unit for normal example is: {}.".format(example))

The expected unit for normal example is: 6.800384122098165.


## Returning to Tom Brady and the Problem of NFL Replica Jerseys

We are now looking at two cases. Case 1 has no salvage value and an ideal order size of `33,267`. Case 2 has a salvage value of `$7.00` and an ideal order size of `40,169`. What is the expected profit for each case? In order to calculate that, we will need another function besides calculating expected units short: calculating expected profit.

In [13]:
def calculate_expected_profits(price, salvage, cost, penalty, order_size, mu, sigma):
    exp_us = calculate_expected_units(order_size, mu, sigma)
    return (price-salvage)*mu - ((cost-salvage)*order_size) - ((price-salvage+penalty)*exp_us)

In [14]:
# solving CASE 1:
salvage_value1 = 0
exp_us1 = np.ceil(calculate_expected_units(my_q1, ave_demand, std_demand))
my_exp_prof1 = np.ceil(calculate_expected_profits(unit_price, salvage_value1, \
                unit_cost, 0, my_q1, ave_demand, std_demand))
print("The expected profit without salvage value: {}".format(my_exp_prof1)) 
# $314,550 when using the standard normal table ($26 difference)

The expected profit without salvage value: 314576.0


In [15]:
# Solving CASE 2:
salvage_value2 = 7.00
exp_us2 = np.ceil(calculate_expected_units(my_q2, ave_demand, std_demand))
my_exp_prof2 = np.ceil(calculate_expected_profits(unit_price, salvage_value2, \
                unit_cost, 0, my_q2, ave_demand, std_demand))
print("The expected profit with salvage value: {}".format(my_exp_prof2))
# $362,514 when using the standard normal table ($14 difference)

The expected profit with salvage value: 362500.0


## Extending the Case with Optimization-Based Procurement

Taking this case study further, what would our ideal order size be if we created some sort of risk-sharing contract  based on channel profit maximization? (This will be only covered in CTL.SC2x: Supply Chain Design but it's worth exploring briefly here.) First, let's return to the concepts of excess and shortage costs.

For the manufacturer, excess costs or shortage costs do not exist because they take no risk. The higher the order size, the bigger the manufacturer's profit. (Profit is a linear equation that goes up with the profit margin for every unit of Q).

For the retailer, excess and shortage costs are the same as you see above:

In [16]:
# repeating from marginal cost with salvage

# shortage cost: price - cost
shortage_cost = unit_price - unit_cost
# excess cost: cost - salvage
excess_cost_salvage = unit_cost - salvage_price
# the critical ratio captures the trade-off
critical_ratio_salvage = shortage_cost / (excess_cost_salvage + shortage_cost)
print("The ideal order size with salvage value is still: {}".format(my_q2)) # should give 40,149
print("The expected profit for the retailer with salvage value is: {}".format(my_exp_prof2))

The ideal order size with salvage value is still: 40149.0
The expected profit for the retailer with salvage value is: 362500.0


At this level, the channel makes the combination of the retailer's profit and the manufacturer's profit combined:

In [17]:
# new variable: the manufacturer's cost
base_cost = 2.90

man_exp_prof1 = my_q2 * (unit_cost - base_cost)
print("The ideal order size is: {}".format(my_q2))
print("The retailer's profit is: {}".format(my_exp_prof2))
print("The manufacturer's profit is: {}".format(man_exp_prof1))
print("Total channel profit is: {}".format(my_exp_prof2 + man_exp_prof1))

The ideal order size is: 40149.0
The retailer's profit is: 362500.0
The manufacturer's profit is: 321192.0
Total channel profit is: 683692.0


For the whole channel, the shortage cost is the price that product sells for minus the lowest cost in the channel (the manufacturer's cost). The excess cost is still the same.

In [18]:
# shortage cost for now
shortage_cost_channel = unit_price - base_cost
# excess cost for now
excess_cost_salvage = unit_cost - salvage_price
# the critical ratio captures the trade-off
critical_ratio_salvage_channel = shortage_cost_channel / (excess_cost_salvage + shortage_cost_channel)

# finding ideal order size
my_q3 = np.ceil(norm.ppf(critical_ratio_salvage_channel, ave_demand, std_demand))
print("The ideal order size with salvage value for the whole channel is: {}".format(my_q3))

# finding expected profits for the channel
my_exp_prof3 = np.ceil(calculate_expected_profits(unit_price, salvage_value2, \
                unit_cost, 0, my_q3, ave_demand, std_demand))
print("The expected profit of the retailer is lower: {}".format(my_exp_prof3))

# finding manufacturer profits at this level
man_exp_prof2 = my_q3 * (unit_cost - base_cost)
print("Total channel profit is: {}".format(man_exp_prof2+my_exp_prof3))

The ideal order size with salvage value for the whole channel is: 43122.0
The expected profit of the retailer is lower: 360571.0
Total channel profit is: 705547.0


That's `$21,855` higher than before! However, the retailer now makes less of a profit. So how can the manufacturer convince the retailer to move towards this higher order size? By proposing one of the following:
* buyback contract;
* revenue sharing contract; or
* options. <br>
We will only explore buyback contracts here -- for the others, see the lectures by Professor Yossi Sheffi.

To find the ideal buyback contract price, use the formula below. The ideal order size and expected channel profit will not change, however, the profit will be split differently between the retailer and the manufacturer, resulting in `$75,541` higher profit for the retailer!

In [21]:
# buyback contract
buyback_price = (((unit_price - salvage_price) / (unit_price - base_cost)) * unit_cost) \
                - ((unit_price*(base_cost - salvage_price)) / (unit_price - base_cost))

my_exp_prof4 = np.ceil(calculate_expected_profits(unit_price, buyback_price, \
                unit_cost, 0, my_q3, ave_demand, std_demand))

print("The optimal buyback price is {}".format(round(buyback_price, 2)))
print("The ideal order size with salvage value for the whole channel is: {}".format(my_q3))
print("Total channel profit is: {}".format(man_exp_prof2+my_exp_prof4))
print("The retailer's profit is now: {}".format(my_exp_prof4)) # previously it was $362,500
print("The manufacturer's profit is now: {}".format(man_exp_prof2)) # previously it was $321,192

The optimal buyback price is 13.45
The ideal order size with salvage value for the whole channel is: 43122.0
Total channel profit is: 783017.0
The retailer's profit is now: 438041.0
The manufacturer's profit is now: 344976.0


This is win-win! Our retailer is now making `$75,541` and our manufacturer is making `$23,784` more in profits.